In [ ]:
# Install necessary requirements

# If you run this notebook on Google Colab, or in standalone mode, you need to install the required packages.
# Uncomment the following lines:

# !pip install choice-learn

# If you run the notebook within the GitHub repository, you need to run the following lines, that can skipped otherwise:
import os
import sys

os.environ["CUDA_VISIBLE_DEVICES"] = ""
sys.path.append("../../")

In [ ]:

import numpy as np
import tensorflow as tf

# Enabling eager execution sometimes decreases fitting time
tf.compat.v1.enable_eager_execution()

In [ ]:
from choice_learn.models import ConditionalLogit

In [ ]:
from choice_learn.datasets import load_swissmetro

swiss_dataset = load_swissmetro(preprocessing="tutorial")
print(swiss_dataset.summary())

In [ ]:
# Initialization of the model
swiss_model = ConditionalLogit(optimizer="Adam", epochs=25, lr=0.01)

# Intercept for train & sm
swiss_model.add_coefficients(feature_name="intercept", items_indexes=[0, 1])
# beta_he for train & sm
swiss_model.add_coefficients(feature_name="headway",
                             items_indexes=[0, 1],
                             coefficient_name="beta_he")
# beta_co for all items
swiss_model.add_coefficients(feature_name="cost",
                             items_indexes=[0, 1, 2])
# beta first_class for train
swiss_model.add_coefficients(feature_name="regular_class",
                             items_indexes=[0])
# beta seats for train
swiss_model.add_coefficients(feature_name="seats", items_indexes=[1])
# betas luggage for car
swiss_model.add_coefficients(feature_name="single_luggage_piece",
                             items_indexes=[2],
                             coefficient_name="beta_luggage=1")
swiss_model.add_coefficients(feature_name="multiple_luggage_piece",
                             items_indexes=[2],
                             coefficient_name="beta_luggage>1")
# beta TT only for car
swiss_model.add_coefficients(feature_name="travel_time",
                             items_indexes=[2],
                             coefficient_name="beta_tt_car")

# betas TT and HE shared by train and sm
swiss_model.add_shared_coefficient(feature_name="travel_time",
                                   items_indexes=[0, 1])
swiss_model.add_shared_coefficient(feature_name="train_survey",
                                   items_indexes=[0, 1],
                                   coefficient_name="beta_survey")


In [ ]:
# Estimation of the model
history = swiss_model.fit(swiss_dataset, get_report=False)

In [ ]:
isinstance(swiss_model.optimizer.get_config()["learning_rate"], np.float32), isinstance(swiss_model.optimizer.get_config()["learning_rate"], np.ndarray)

In [ ]:
swiss_model.save_model("test_save")

In [ ]:
swiss_model2 = ConditionalLogit.load_model("test_save")

In [ ]:
hist = swiss_model2.fit(swiss_dataset)

In [ ]:
import shutil

shutil.rmtree("test_save")

## Save every n epochs with a custom tf.Callback

In [ ]:
class SaveCallback(tf.keras.callbacks.Callback):
    """Callback to save regularly the model during training."""

    def __init__(self, base_dir, save_every_n, *args, **kwargs):
        """Instantiate callback."""
        self.base_dir = base_dir
        self.save_every_n = save_every_n
        super().__init__(*args, **kwargs)

    def on_epoch_end(self, epoch, logs=None):
        """Define saving at the end of each epoch."""
        _ = logs
        if (epoch + 1) % self.save_every_n == 0:
            self._save_model(epoch=epoch)

    def _save_model(self, epoch):
        """Handle model saving internally."""
        dirname = os.path.join(self.base_dir, f"epoch_{epoch}")
        self.model.save_model(dirname)

In [ ]:
# Initialization of the model
swiss_model = ConditionalLogit(optimizer="Adam", epochs=25, lr=0.01, callbacks=[SaveCallback(base_dir="test_save_cb", save_every_n=2)])

# Intercept for train & sm
swiss_model.add_coefficients(feature_name="intercept", items_indexes=[0, 1])
# beta_he for train & sm
swiss_model.add_coefficients(feature_name="headway",
                             items_indexes=[0, 1],
                             coefficient_name="beta_he")
# beta_co for all items
swiss_model.add_coefficients(feature_name="cost",
                             items_indexes=[0, 1, 2])
# beta first_class for train
swiss_model.add_coefficients(feature_name="regular_class",
                             items_indexes=[0])
# beta seats for train
swiss_model.add_coefficients(feature_name="seats", items_indexes=[1])
# betas luggage for car
swiss_model.add_coefficients(feature_name="single_luggage_piece",
                             items_indexes=[2],
                             coefficient_name="beta_luggage=1")
swiss_model.add_coefficients(feature_name="multiple_luggage_piece",
                             items_indexes=[2],
                             coefficient_name="beta_luggage>1")
# beta TT only for car
swiss_model.add_coefficients(feature_name="travel_time",
                             items_indexes=[2],
                             coefficient_name="beta_tt_car")

# betas TT and HE shared by train and sm
swiss_model.add_shared_coefficient(feature_name="travel_time",
                                   items_indexes=[0, 1])
swiss_model.add_shared_coefficient(feature_name="train_survey",
                                   items_indexes=[0, 1],
                                   coefficient_name="beta_survey")


In [ ]:

# Estimation of the model
history = swiss_model.fit(swiss_dataset, get_report=True)

In [ ]:
# remove
shutil.rmtree("test_save_cb")